In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data= pd.read_csv('Churn_Modelling.csv')

data.info()
data.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [3]:
X= data.iloc[:,3:13]
y= data.iloc[:, 13]

X.Geography.unique()
X['Gender'].unique()

geo_cat= pd.get_dummies(X['Geography'], drop_first= True)
gender_cat= pd.get_dummies(X['Gender'], drop_first= True)

X= pd.concat([X, geo_cat, gender_cat], axis=1)

X= X.drop(['Geography', 'Gender'], axis=1)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X,y, test_size = 0.2 , random_state = 9)

In [5]:
from sklearn.preprocessing import StandardScaler

sc= StandardScaler()

X_train= sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


Perform Hyperparameter Optimization


In [6]:
pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in c:\users\user\anaconda3\lib\site-packages (2.5.0)



In [7]:
import tensorflow as tf

In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, BatchNormalization
from keras.layers import LeakyReLU, PReLU, ELU
from keras.layers import Dropout
from keras.activations import relu, sigmoid

In [9]:
def create_model(layers, activation):
    model = Sequential()
    
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, input_dim = X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))
    
    model.compile(optimizer ='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [10]:
model= KerasClassifier(build_fn=create_model, verbose=0)

In [11]:
layers = [(20,), (40,20), (45,30,15)] 

In [12]:
activations = ['sigmoid', 'relu']

param_grid =dict(layers = layers, activation = activations, batch_size = [128, 256], epochs = [30])

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5)

In [13]:
grid_result= grid.fit( X_train, y_train)

Best Model


In [14]:
grid_result.best_score_, grid_result.best_params_


(0.8565000176429749,
 {'activation': 'relu',
  'batch_size': 128,
  'epochs': 30,
  'layers': (45, 30, 15)})

In [15]:
y_pred = grid.predict(X_test)
y_pred = (y_pred > 0.5)

C:\Users\user\anaconda3\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1535,   43],
       [ 242,  180]], dtype=int64)

In [17]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)

score

0.8575

Deep Learning Model

In [18]:
X=data.iloc[:,3:13]
y=data.iloc[:,3:13]

In [19]:
geography=pd.get_dummies(X['Geography'],drop_first= True)
gender= pd.get_dummies(X['Gender'],drop_first= True)


In [20]:
X = X.drop(['Geography', 'Gender'], axis=1)
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [21]:
X=pd.concat([X, geography, gender], axis=1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test , y_train, y_test= train_test_split(X,y, test_size=0.2 , random_state=0)


In [23]:
from sklearn.preprocessing import StandardScaler

sc= StandardScaler()

X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [24]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU, PReLU, ELU
from keras.layers import Dropout

In [25]:
classifier = Sequential()

classifier.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu',input_dim=11))
classifier.add(Dropout(0.3))
classifier.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(units=10,kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='swish'))

In [26]:
classifier.summary()

Model: "sequential_61"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_184 (Dense)            (None, 10)                120       
_________________________________________________________________
dropout_123 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_185 (Dense)            (None, 10)                110       
_________________________________________________________________
dropout_124 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_186 (Dense)            (None, 10)                110       
_________________________________________________________________
dropout_125 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_187 (Dense)            (None, 1)               

In [27]:
classifier.compile(optimizer='adam', loss='binary.crossentropy',metrics=['accuracy'])

In [28]:
tf.keras.callbacks.History()

In [29]:
tf.convert_to_tensor

<function tensorflow.python.framework.ops.convert_to_tensor_v2_with_dispatch(value, dtype=None, dtype_hint=None, name=None)>

In [32]:
import numpy as np

X = np.asarray(X).astype(np.float32)

In [35]:
X_train = np.asarray(X_train).astype(np.float32)

In [39]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for element in dataset.as_numpy_iterator():
  print(element)

1
2
3


In [41]:
as_numpy_iterator()

NameError: name 'as_numpy_iterator' is not defined

In [40]:
model_history=classifier.fit(X_train,y_train,validation_split=0.33,batch_size=10,epochs=100,verbose=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [53]:
model_history.history.keys()

NameError: name 'model_history' is not defined